In [3]:
import re
import pandas as pd
import win32com.client
import pickle
import os
import glob
import email

In [4]:
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")

## Finding the Email folder you wish to scrape

In [5]:
for i in range(25):
    try:
        box = outlook.GetDefaultFolder(i)
        name = box.Name
        print(i, name)
    except:
        pass

3 Deleted Items
4 Outbox
5 Sent Items
6 Inbox
9 Calendar
10 Contacts
11 Journal
12 Notes
13 Tasks
14 Reminders
15 Reminders
16 Drafts
17 Conflicts
19 Conflicts
20 Sync Issues
21 Local Failures
22 Server Failures
23 Junk Email


## Scraping Email Content Directly

In [6]:
inbox = outlook.GetDefaultFolder(6) # "6" refers to the index of a folder - in this case,
                                    # the inbox. You can change that number to reference
                                    # any other folder
messages = inbox.Items
message = messages.GetLast()
body_content = message.body

# print (body_content) #If you want to check the body of the most recent message

In [7]:
#note that this cell searches the bdy of an email
#you must change if you want to view the actual subject  / sender details

emails = []
for message in messages:
    if 'block payment' in message.body.lower(): 
        emails.append( (message.body) )

In [8]:
#emails[0] #prints the most recent email from your search

In [10]:
email_concat = pd.DataFrame()
for i in emails:
    dic={}
    dic['Currency'] = re.findall('([U][S][D]|[G][B][P]|[E][U][R]|[J][P][Y]|[Z][A][R]|[A][U][D]|[C][A][D]|[S][G][D])' , i[i.find('CM')-4:])
    dic['Amount'] = [i.strip() for i in re.findall('([\s\w\d\s][\s\d\w][,\w\d\s][0-9\w\d\s][0-9][0-9][.][0-9][0-9])' , i[i.find('CM')-4:]) ]
    dic['Date'] =  re.findall('[0-9][0-9][/][0-9][0-9][/][0-9][0-9]' , i[i.find('CM')-4:])

    #scraping the body of an email directly into a dataframe, below checks that each array is the same length
    count = []
    for x in dic.keys():
        count.append( len(dic[x]))
    if len(set(count)) == 1:
        print ('ok')
    else:
        print ('Problem' + str ( re.findall( '[S][e][n][t][:][ ]([ 0-9 A-Z a-z :]+)' , i ) ) \
               + str ( re.findall( '[S][u][b][j][e][c][t][:][ ]([0-9A-Za-z :,.]+)' , i ) ) )
        
    
    email = pd.DataFrame(data =  dic)
    email_concat = pd.concat([email_concat , email]) 

    #print(dic)

ok
ok
ok
ok
ok
ok
ok


## Savings Attachments from an email

In [11]:
#As mentioned above, we are now scraping the subject of emails.
for i in messages:
    if len (re.findall('[S][u][b][j][e][c][t][:][ ][Aa]' , message.subject)) > 0:
            print(message.body)

In [14]:
get_path = r"enter_here"

#following loop scrapes and downloads all of the attachments based on the Subject containing AON.

emails2 = []
for message in messages:
    if len(re.findall('[A][o][n]' , message.subject)) > 0 : 
        emails2.append( (message.body))
        #print('appended' + str(message.subject))
        attachments = message.Attachments
        #print('Length of attachments '+len(attachments))
        counter = len(attachments)
        countercheck = 0
        for i in range(0, len(attachments)+2):
            try:
                attachment = attachments.Item(i)
                attachment.SaveASFile(os.path.join(get_path, attachment.FileName[-30:])) #Saves to the attachment to current folder
                #print(i)
                #print (attachment)
                countercheck+=1
            except:
                #print
                #print(str(i) +' didnt work ')
                pass
        if counter != countercheck:
            #print ('PROBLEM WITH THIS EMAIL !' + str(message.subject))
            pass

## Opening up .msg files, then downloading their attachments (repeatable) should put in a function

In [16]:
## Once these attachments have been downloaded
#we can open them up as .msg files and then keep reopening them until we get the attachment we want.  
level_one_emails = r'path here'
level_two_path = get_path + "\Level 2"
for i in level_one_emails[1:]:
    msg = outlook.OpenSharedItem(i)
    count_attachments = msg.Attachments.Count
    if count_attachments > 0:
        for item in range(count_attachments):
            attachment = msg.Attachments.Item(item + 1)
            print (msg.Attachments.Item(item + 1).Filename)
            attachment.SaveASFile(os.path.join(level_two_path, attachment.FileName[-100:]))

SyntaxError: EOL while scanning string literal (<ipython-input-16-1475e9b1e19b>, line 3)

In [17]:
level_three_path = r"path_here"
for i in level_two_emails[1:]:
    msg = outlook.OpenSharedItem(i)
    count_attachments = msg.Attachments.Count
    if count_attachments > 0:
        for item in range(count_attachments):
            attachment = msg.Attachments.Item(item + 1)
            print (msg.Attachments.Item(item + 1).Filename)
            attachment.SaveASFile(os.path.join(level_three_path, attachment.FileName[-100:]))

NameError: name 'level_two_emails' is not defined

## Opening up .eml files and downloading their attachments

In [18]:
#This doesnt work for .msg attachments for some reason
path = r"path_here"

listing = os.listdir(path)

for file in listing:
    
    if str.lower(file[-3:])=="eml":
        msg = email.message_from_file(open(path + '\\'+ file))
        attachments = msg.get_payload()
        
        for attachment in attachments[1:]: #don't know why but there seems to be an empty attachment at the beginning of each payload
            
            try:
                filename = attachment.get_filename()
                #print(filename)
                f = open(os.path.join(path + '\\Level 2\\'+filename), 'wb').write(attachment.get_payload(decode=True))
                #f.close()
            
            except Exception as detail:
                #print('Error: {}'.format(file))
                #print (detail)


SyntaxError: unexpected EOF while parsing (<ipython-input-18-01a49cba1243>, line 22)